# Build a product recommendation engine

![](https://raw.githubusercontent.com/IBM/product-recommendation-with-watson-ml/master/doc/source/images/shopping.png)

This notebook contains steps and code to create a recommendation engine based on shopping history, deploy that model to Watson Machine Learning, and create an app with PixieApps. This notebook runs on Python 3.x with Apache Spark 2.1.

## Learning Goals

The learning goals of this notebook are:

* Load a CSV file into the Object Storage service linked to your Watson Studio
* Use the *k*-means algorithm, which is useful for cluster analysis in data mining, to segment customers into clusters for the purpose of making an in-store purchase recommendation
* Deploy the model to the IBM Watson Machine Learning service in IBM Cloud to create your recommendation application

## Table of contents

1. [Setup](#setup)<br>
2. [Load and explore data](#load)<br>
3. [Create a KMeans model](#kmeans)<br>
   3.1. [Prepare data](#prepare_data)<br>
   3.2. [Create clusters and define the model](#build_model)<br>
4. [Persist the model](#persist)<br>
5. [Deploy the model to the cloud](#deploy)<br>
   5.1. [Create deployment for the model](#create_deploy)<br>
   5.2. [Test model deployment](#test_deploy)<br>
6. [Create product recommendations](#create_recomm)<br>
   6.1. [Test product recommendations model](#test_recomm)<br>
7. [Summary and next steps](#summary)<br>

## 1. Setup


Before you use the sample code in this notebook, you must perform the following setup tasks:

* Create a Watson Machine Learning service instance (a free plan is offered) and associate it with your project


We'll be using a few libraries for this exercise:

1. [Watson Machine Learning Client](http://wml-api-pyclient.mybluemix.net/): Client library to work with the Watson Machine Learning service on IBM Cloud. Library available on [pypi](https://pypi.org/project/watson-machine-learning-client/). Service available on [IBM Cloud](https://cloud.ibm.com/catalog/services/machine-learning).
1. [Pixiedust](https://github.com/pixiedust/pixiedust): Python Helper library for Jupyter Notebooks. Available on [pypi](https://pypi.org/project/pixiedust/).
1. [ibmos2spark](https://github.com/ibm-watson-data-lab/ibmos2spark): Facilitates Data I/O between Spark and IBM Object Storage services

In [1]:
!pip install --upgrade ibmos2spark
!pip install --upgrade pixiedust
!pip install --upgrade watson-machine-learning-client

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190902092133-0002
KERNEL_ID = 28398ee7-a048-4d4b-8367-9b3bc67efc63
  Using cached https://files.pythonhosted.org/packages/c6/81/1edb24382edef1ca636e87972b2da286b8271a586c728a21f916d3cd76cd/ibmos2spark-1.0.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c0/4e/fd492e91abdc2d2fcb70ef453064d980688762079397f779758e055f6575/Markdown-3.1.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ec/be/5ab8abdd8663c0386ec2dd595a5bc0e23330a0549b8a91e32f38c20845b6/lxml-4.4.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/2e/37/5dd0dd89b87bb5f0f32a7e775458412c52d78f230ab8d0c65df6aabc4479/astunparse-1.6.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e4/8d/9e28e9af95739e6d2d2f8d4bef0b3432da40b7c3588fbad4298c1be09e48/geojson-2.5.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.

In [2]:
!pip install watson-machine-learning-client

  Using cached https://files.pythonhosted.org/packages/0e/a1/c503614455fb734b0989e8d6abaf24d0544d7370f7eb2b80ffbc99a40caf/watson_machine_learning_client-1.0.371-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0f/b1/02eebed49c754b01b17de7705caa8c4ceecfb4f926cdafc220c863584360/lomond-0.3.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/73/9b/52e228545d14f14bb2a1622e225f38463c8726645165e1cb7dde95bfe6d4/pandas-0.25.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/69/1b/b853c7a9d4f6a6d00749e94eb6f3a041e342a885b87340b79c1ef73e3a78/certifi-2019.6.16-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/dc/88/d3213e2f3492daf09d8b41631ad6899f56db17ce83ea9c8a579902bafe5e/tqdm-4.35.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e6/60/247f23a7121ae632d62811ba7f273d0e58972d75e58a94d329d51550a47d/urllib3-1.25.3-py2.py3-none-any.whl
  Using cached https://

In [3]:
import pixiedust

Pixiedust database opened successfully


<a id="load"></a>
## 2. Load and explore data

In this section you will load the data file that contains the customer shopping data using PixieDust's [`sampleData`](https://pixiedust.github.io/pixiedust/loaddata.html) method:

In [4]:
df = pixiedust.sampleData('https://raw.githubusercontent.com/IBM/product-recommendation-with-watson-ml/master/data/customers_orders1_opt.csv')

Downloaded 5648773 bytes
Creating pySpark DataFrame for 'https://raw.githubusercontent.com/IBM/product-recommendation-with-watson-ml/master/data/customers_orders1_opt.csv'. Please wait...
Loading file using 'SparkSession'
Successfully created pySpark DataFrame for 'https://raw.githubusercontent.com/IBM/product-recommendation-with-watson-ml/master/data/customers_orders1_opt.csv'


To better examine and visualize the data, run the following cell to view it in a table format. Note that Pixiedust's `display` method can also render data using various chart types, such as pie charts, line graphs, and scatter plots.

In [ ]:
display(df)

CUSTNAME,GenderCode,ADDRESS1,CITY,STATE,COUNTRY_CODE,POSTAL_CODE,POSTAL_CODE_PLUS4,ADDRESS2,EMAIL_ADDRESS,PHONE_NUMBER,CREDITCARD_TYPE,LOCALITY,SALESMAN_ID,NATIONALITY,NATIONAL_ID,CREDITCARD_NUMBER,DRIVER_LICENSE,CUST_ID,ORDER_ID,ORDER_DATE,ORDER_TIME,FREIGHT_CHARGES,ORDER_SALESMAN,ORDER_POSTED_DATE,ORDER_SHIP_DATE,AGE,ORDER_VALUE,T_TYPE,PURCHASE_TOUCHPOINT,PURCHASE_STATUS,ORDER_TYPE,GENERATION,Baby Food,Diapers,Formula,Lotion,Baby wash,Wipes,Fresh Fruits,Fresh Vegetables,Beer,Wine,Club Soda,Sports Drink,Chips,Popcorn,Oatmeal,Medicines,Canned Foods,Cigarettes,Cheese,Cleaning Products,Condiments,Frozen Foods,Kitchen Items,Meat,Office Supplies,Personal Care,Pet Supplies,Sea Food,Spices
Calvin Shupe,Mr.,3335 Willow Oaks Lane,Albosaggia,SO,IT,23100,0,None,Calvin.M.Shupe@spambob.com,0331 4473271,VISA,None,NE373,IT,RHFGBS20R46E848A,4308565230784723,None,10047,529,2016-05-19 00:00:00,2016-05-19 20:47:39.380000,9.0,SC388,2016-06-02 00:00:00,15/06/2016,49,45.66,Complete,Phone,Occasional,LowValue,Gen_X,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Harold Magee,Mr.,565 Cinnamon Lane,Ashland,VA,US,23005,0,None,Harold.A.Magee@mailinator.com,503-308-1831,American Express,None,NW310,UK,YS731666A,375517931599425,None,10155,1267,2016-06-09 00:00:00,2016-06-09 18:50:44.886000,21.97,WE223,2016-06-28 00:00:00,10/07/2016,40,22.38,Complete,Phone,Occasional,LowValue,Gen_Y,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Mandy Fernandez,Mrs.,4103 Wilmar Farm Road,Banora Point,NSW,AU,2486,0,None,Mandy.M.Fernandez@pookmail.com,(03) 5361 9459,Master Card,None,WE355,ES,2359270E,5167671230495638,None,10239,5977,2016-06-11 00:00:00,2016-06-11 15:36:44.886000,9.51,RP319,2016-07-10 00:00:00,20/07/2016,42,32.99,Complete,Phone,Occasional,LowValue,Gen_X,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
Robert Ojeda,Mr.,546 Lakewood Drive,Belleville,IL,US,62220,0,None,Robert.M.Ojeda@mailinator.com,503-705-8957,JCB,None,NC361,FR,2.29E+14,3528362337592811,None,10275,1998,2016-01-24 00:00:00,2016-01-24 23:13:54.564000,18.24,SE400,2016-02-26 00:00:00,07/03/2016,76,236.03,Complete,Desktop,Occasional,HighValue,Baby_Boomers,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1
Jennifer Becker,Mrs.,4374 Lindale Avenue,Bloomington,IL,US,61701,0,None,Jennifer.D.Becker@spambob.com,803-223-5484,JCB,None,SE337,FR,1.69E+14,3528310310507322,None,10351,3408,2016-01-06 00:00:00,2016-01-06 18:23:54.564000,28.55,NW118,2016-01-26 00:00:00,07/02/2016,24,299.35,Complete,Phone,Frequent,HighValue,Gen_Y,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,1,0
James Armstrong,Mr.,4544 Hidden Valley Road,Bobigny,None,FR,93000,0,None,James.V.Armstrong@pookmail.com,05.78.34.01.73,VISA,None,SC262,UK,MA467934D,4448798497513607,None,10359,6038,2016-06-09 00:00:00,2016-06-09 18:50:44.886000,38.99,WE223,2016-06-28 00:00:00,10/07/2016,67,206.45,Cancelled,Desktop,Frequent,HighValue,Baby_Boomers,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1
Joe Stone,Mr.,3664 Bicetown Road,Bridgeville,PA,US,15017,0,None,Joe.N.Stone@dodgeit.com,928-446-4014,JCB,None,NW112,IT,XGDCLZ97B50A681W,3528322748268476,None,10439,8797,2016-11-11 00:00:00,2016-11-11 18:24:36.250000,7.85,SC328,2016-12-13 00:00:00,20/12/2016,23,110.05,Cancelled,Phone,Occasional,MediumValue,Gen_Y,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0
Kay Fauver,Mrs.,2932 Despard Street,Campello Sul Clitunno,PG,IT,6042,0,None,Kay.R.Fauver@trashymail.com,0382 7527732,Diners Club,None,SC199,IT,EXXYCO23R26B670B,36901578474238,None,10543,1311,2016-11-03 00:00:00,2016-11-03 19:43:54.804000,7.85,NC364,2016-12-04 00:00:00,12/12/2016,58,23.07,Complete,Phone,Occasional,LowValue,Baby_Boomers,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
Aaron Gossage,Mr.,2572 Badger Pond Lane,Cessaniti,VV,IT,89816,0,None,Aaron.N.Gossage@pookmail.com,0321 9690752,Discover,None,NW187,ES,8189350Q,6011106136740411,None,10683,3148,2016-04-26 00:00:00,2016-04-26 02:22:36.564000,9.22,NE238,2016-05-05 00:00:00,19/05/2016,a

<a id="kmeans"></a>
## 3. Create a *k*-means model with Spark

In this section of the notebook you use the *k*-means implementation to associate every customer to a cluster based on their shopping history.

First, import the Apache Spark Machine Learning packages ([MLlib](http://spark.apache.org/docs/2.2.0/api/python/pyspark.ml.html)) that you need in the subsequent steps:


In [6]:
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.ml.clustering import KMeansModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors

<a id="prepare_data"></a>
### 3.1 Prepare data

Create a new data set with just the data that you need. Filter the columns that you want, in this case the customer ID column and the product-related columns. Remove the columns that you don't need for aggregating the data and training the model:

In [7]:
# Here are the product cols. In a real world scenario we would query a product table, or similar.
product_cols = ['Baby Food', 'Diapers', 'Formula', 'Lotion', 'Baby wash', 'Wipes', 'Fresh Fruits', 'Fresh Vegetables', 'Beer', 'Wine', 'Club Soda', 'Sports Drink', 'Chips', 'Popcorn', 'Oatmeal', 'Medicines', 'Canned Foods', 'Cigarettes', 'Cheese', 'Cleaning Products', 'Condiments', 'Frozen Foods', 'Kitchen Items', 'Meat', 'Office Supplies', 'Personal Care', 'Pet Supplies', 'Sea Food', 'Spices']
# Here we get the customer ID and the products they purchased
df_filtered = df.select(['CUST_ID'] + product_cols)

Run the `display()` command again, this time to view the filtered information:

In [ ]:
display(df_filtered)

CUST_ID,Baby Food,Diapers,Formula,Lotion,Baby wash,Wipes,Fresh Fruits,Fresh Vegetables,Beer,Wine,Club Soda,Sports Drink,Chips,Popcorn,Oatmeal,Medicines,Canned Foods,Cigarettes,Cheese,Cleaning Products,Condiments,Frozen Foods,Kitchen Items,Meat,Office Supplies,Personal Care,Pet Supplies,Sea Food,Spices
10015,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
10059,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0
10123,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
10147,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
10223,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10347,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
10439,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0
10455,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10555,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1
10567,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0


Now, aggregate the individual transactions for each customer to get a single score per product, per customer.

In [ ]:
df_customer_products = df_filtered.groupby('CUST_ID').sum()  # Use customer IDs to group transactions by customer and sum them up
df_customer_products = df_customer_products.drop('sum(CUST_ID)')
display(df_customer_products)

CUST_ID,sum(Baby Food),sum(Diapers),sum(Formula),sum(Lotion),sum(Baby wash),sum(Wipes),sum(Fresh Fruits),sum(Fresh Vegetables),sum(Beer),sum(Wine),sum(Club Soda),sum(Sports Drink),sum(Chips),sum(Popcorn),sum(Oatmeal),sum(Medicines),sum(Canned Foods),sum(Cigarettes),sum(Cheese),sum(Cleaning Products),sum(Condiments),sum(Frozen Foods),sum(Kitchen Items),sum(Meat),sum(Office Supplies),sum(Personal Care),sum(Pet Supplies),sum(Sea Food),sum(Spices)
12940,2,0,1,1,0,1,1,0,0,1,0,0,0,1,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0
13285,0,0,1,0,0,0,0,1,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11858,0,0,0,0,0,0,0,1,0,0,1,0,1,2,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0
13623,0,0,0,0,0,0,0,0,0,0,2,0,2,2,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0
13508,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,0,1
14944,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,1,1,1,1,0,1
11143,0,0,0,0,0,0,0,0,0,0,2,0,2,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0
14846,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
12436,0,0,0,1,1,0,0,0,0,0,4,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
12014,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


<a id="build_model"></a>
### 3.2 Create clusters and define the model 

Create 100 clusters with a *k*-means model based on the number of times a specific customer purchased a product.

| No Clustering | Clustering |
|------|------|
|  ![](https://raw.githubusercontent.com/IBM/product-recommendation-with-watson-ml/master/doc/source/images/kmeans-1.jpg)  | ![](https://raw.githubusercontent.com/IBM/product-recommendation-with-watson-ml/master/doc/source/images/kmeans-2.jpg) |

First, create a feature vector by combining the product and quantity columns:

In [10]:
assembler = VectorAssembler(inputCols=["sum({})".format(x) for x in product_cols],outputCol="features") # Assemble vectors using product fields

Next, create the *k*-means clusters and the pipeline to define the model:

In [11]:
kmeans = KMeans(maxIter=50, predictionCol="cluster").setK(100).setSeed(1)  # Initialize model
pipeline = Pipeline(stages=[assembler, kmeans])
model = pipeline.fit(df_customer_products)

Finally, calculate the cluster for each customer by running the original dataset against the *k*-means model: 

In [ ]:
df_customer_products_cluster = model.transform(df_customer_products)
display(df_customer_products_cluster)

CUST_ID,sum(Baby Food),sum(Diapers),sum(Formula),sum(Lotion),sum(Baby wash),sum(Wipes),sum(Fresh Fruits),sum(Fresh Vegetables),sum(Beer),sum(Wine),sum(Club Soda),sum(Sports Drink),sum(Chips),sum(Popcorn),sum(Oatmeal),sum(Medicines),sum(Canned Foods),sum(Cigarettes),sum(Cheese),sum(Cleaning Products),sum(Condiments),sum(Frozen Foods),sum(Kitchen Items),sum(Meat),sum(Office Supplies),sum(Personal Care),sum(Pet Supplies),sum(Sea Food),sum(Spices),features,cluster
11458,0,1,1,1,0,1,0,0,1,2,3,0,2,2,0,1,2,3,3,0,2,0,0,0,0,0,0,2,3,"(29,[1,2,3,5,8,9,10,12,13,15,16,17,18,20,27,28],[1.0,1.0,1.0,1.0,1.0,2.0,3.0,2.0,2.0,1.0,2.0,3.0,3.0,2.0,2.0,3.0])",9
10914,0,0,0,1,0,1,1,0,0,0,1,1,1,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,"(29,[3,5,6,10,11,12,16,20],[1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0])",80
14514,0,0,0,0,0,0,0,0,0,0,2,1,2,2,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,"(29,[10,11,12,13,16,20],[2.0,1.0,2.0,2.0,2.0,1.0])",44
15173,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1,"(29,[17,18,21,28],[1.0,1.0,1.0,1.0])",59
13483,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,"(29,[10,12,16,20],[1.0,1.0,1.0,1.0])",80
13009,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,"(29,[19,26],[1.0,1.0])",35
11146,0,0,0,1,1,0,0,0,0,0,3,0,3,1,0,1,3,1,1,0,1,1,1,0,1,0,0,0,0,"(29,[3,4,10,12,13,15,16,17,18,20,21,22,24],[1.0,1.0,3.0,3.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0])",79
12284,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1,"(29,[9,15,25,27,28],[1.0,1.0,1.0,1.0,1.0])",19
15221,1,1,1,0,0,1,1,0,1,2,0,1,0,0,0,2,1,1,2,0,1,0,0,0,0,2,0,0,3,"(29,[0,1,2,5,6,8,9,11,15,16,17,18,20,25,28],[1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,3.0])",62
11243,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,3,0,2,2,0,0,0,0,0,1,2,0,3,3,"(29,[9,15,17,18,24,25,27,28],[4.0,3.0,2.0,2.0,1.0,2.0,3.0,3.0])",13


<a id="persist"></a>
## 4. Persist the model 

In this section you will learn how to store your pipeline and model in Watson Machine Learning repository by using Python client libraries.

### 4.1 Configure IBM Watson Machine Learning credentials

To access your machine learning repository programmatically, you need to copy in your credentials, which you can see in your **IBM Watson Machine Learning** service details in IBM Cloud.

> **IMPORTANT**: Update `apikey` and `instance_id` below. Credentials can be found on _Service Credentials_ tab of the Watson Machine Learning service instance created on the IBM Cloud.

In [13]:
# The code was removed by Watson Studio for sharing.

{'apikey': 'f6-wpCTDJAqEGwroHLReUZbDBfaQwvvM6VXgOwesYYvG', 'iam_apikey_description': 'Auto-generated for key 4da7e2a1-41cc-4792-a8de-33ffabf98b64', 'iam_apikey_name': 'wdp-writer', 'iam_role_crn': 'crn:v1:bluemix:public:iam::::serviceRole:Writer', 'iam_serviceid_crn': 'crn:v1:bluemix:public:iam-identity::a/37558c3ea5034b34a2014e336f80fe3a::serviceid:ServiceId-b42ecdf2-990f-4818-8d6c-c8807311e0b9', 'instance_id': 'ddf95799-ca07-43d1-8015-e3c44711753e', 'url': 'https://jp-tok.ml.cloud.ibm.com'}


Connect to the Watson Machine Learning service using the provided credentials.

In [14]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
client = WatsonMachineLearningAPIClient(wml_credentials)
print(client.version)

1.0.371


### 4.2 Save the model 

#### Save the model to the Watson Machine Learning repository

You use the Watson Machine Learning client's [Repository class](http://wml-api-pyclient.mybluemix.net/#repository) to store and manage models in the Watson Machine Learning service. 

> **NOTE**: You can also use Watson Studio to manage models. In this notebook we are using the client library instead.

In [15]:
train_data = df_customer_products.withColumnRenamed('CUST_ID', 'label')

> **TIP**: Update the cell below with your name, email, and name you wish to give to your model.

In [16]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "IBM", 
               client.repository.ModelMetaNames.NAME: "Shopping Recommendation Engine"}
published_model = client.repository.store_model(model=model, pipeline=pipeline, meta_props=model_props, training_data=train_data)

/home/spark/shared/user-libs/python3/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


> **NOTE**: You can delete a model from the repository by calling `client.repository.delete`.

#### Display list of existing models in the Watson Machine Learning repository 

In [17]:
client.repository.list_models()

------------------------------------  ------------------------------  ------------------------  ---------
GUID                                  NAME                            CREATED                   FRAMEWORK
4307cf67-1556-4a64-810b-5b40b04fa3d5  Shopping Recommendation Engine  2019-09-02T09:24:44.226Z  mllib-2.3
------------------------------------  ------------------------------  ------------------------  ---------


#### Display information about the saved model

In [18]:
import json
saved_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(saved_model_uid)
print(json.dumps(model_details, indent=2))

{
  "metadata": {
    "guid": "4307cf67-1556-4a64-810b-5b40b04fa3d5",
    "url": "https://jp-tok.ml.cloud.ibm.com/v3/wml_instances/ddf95799-ca07-43d1-8015-e3c44711753e/published_models/4307cf67-1556-4a64-810b-5b40b04fa3d5",
    "created_at": "2019-09-02T09:24:44.226Z",
    "modified_at": "2019-09-02T09:24:44.274Z"
  },
  "entity": {
    "runtime_environment": "spark-2.3",
    "learning_configuration_url": "https://jp-tok.ml.cloud.ibm.com/v3/wml_instances/ddf95799-ca07-43d1-8015-e3c44711753e/published_models/4307cf67-1556-4a64-810b-5b40b04fa3d5/learning_configuration",
    "author": {
      "name": "IBM"
    },
    "name": "Shopping Recommendation Engine",
    "label_col": "label",
    "learning_iterations_url": "https://jp-tok.ml.cloud.ibm.com/v3/wml_instances/ddf95799-ca07-43d1-8015-e3c44711753e/published_models/4307cf67-1556-4a64-810b-5b40b04fa3d5/learning_iterations",
    "training_data_schema": {
      "fields": [
        {
          "metadata": {
            "modeling_role": "targ

<a id="deploy"></a>
## 5. Deploy model to the IBM cloud

You use the Watson Machine Learning client's [Deployments class](http://wml-api-pyclient.mybluemix.net/#deployments) to deploy and score models.

### 5.1 Create an online deployment for the model


In [19]:
created_deployment = client.deployments.create(saved_model_uid, 'Shopping Recommendation Engine Deployment')



#######################################################################################

Synchronous deployment creation for uid: '4307cf67-1556-4a64-810b-5b40b04fa3d5' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='8a1f054c-03a2-4ced-9d2a-3367fe1bceda'
------------------------------------------------------------------------------------------------




### 5.2 Retrieve the scoring endpoint for this model

In [20]:
scoring_endpoint = client.deployments.get_scoring_url(created_deployment)
print(scoring_endpoint)

https://jp-tok.ml.cloud.ibm.com/v3/wml_instances/ddf95799-ca07-43d1-8015-e3c44711753e/deployments/8a1f054c-03a2-4ced-9d2a-3367fe1bceda/online


<a id="test_deploy"></a>
### 5.3 Test the deployed model

To verify that the model was successfully deployed to the cloud, you'll specify a customer ID, for example customer 12027, to predict this customer's cluster against the Watson Machine Learning deployment, and see if it matches the cluster that was previously associated this customer ID.

In [21]:
customer = df_customer_products_cluster.filter('CUST_ID = 12027').collect()
print("Previously calculated cluster = {}".format(customer[0].cluster))

Previously calculated cluster = 73


To determine the customer's cluster using Watson Machine Learning, you need to load the customer's purchase history. This function uses the local data frame to select every product field and the number of times that customer 12027 purchased a product.

In [22]:
from six import iteritems
def get_product_counts_for_customer(cust_id):
    cust = df_customer_products.filter('CUST_ID = {}'.format(cust_id)).take(1)
    fields = []
    values = []
    for row in customer:
        for product_col in product_cols:
            field = 'sum({})'.format(product_col)
            value = row[field]
            fields.append(field)
            values.append(value)
    return (fields, values)

This function takes the customer's purchase history and calls the scoring endpoint:

In [23]:
def get_cluster_from_watson_ml(fields, values):
    scoring_payload = {'fields': fields, 'values': [values]}
    predictions = client.deployments.score(scoring_endpoint, scoring_payload)   
    return predictions['values'][0][len(product_cols)+1]

Finally, call the functions defined above to get the product history, call the scoring endpoint, and get the cluster associated to customer 12027:

In [24]:
product_counts = get_product_counts_for_customer(12027)
fields = product_counts[0]
values = product_counts[1]
print("Cluster calculated by Watson ML = {}".format(get_cluster_from_watson_ml(fields, values)))

Cluster calculated by Watson ML = 73


<a id="create_recomm"></a>
## 6. Create product recommendations

Now you can create some product recommendations.

First, run this cell to create a function that queries the database and finds the most popular items for a cluster. In this case, the **df_customer_products_cluster** dataframe is the database.

In [25]:
# This function gets the most popular clusters in the cell by grouping by the cluster column
def get_popular_products_in_cluster(cluster):
    df_cluster_products = df_customer_products_cluster.filter('cluster = {}'.format(cluster))
    df_cluster_products_agg = df_cluster_products.groupby('cluster').sum()
    row = df_cluster_products_agg.rdd.collect()[0]
    items = []
    for product_col in product_cols:
        field = 'sum(sum({}))'.format(product_col)
        items.append((product_col, row[field]))
    sortedItems = sorted(items, key=lambda x: x[1], reverse=True) # Sort by score
    popular = [x for x in sortedItems if x[1] > 0]
    return popular

Now, run this cell to create a function that will calculate the recommendations based on a given cluster. This function finds the most popular products in the cluster, filters out products already purchased by the customer or currently in the customer's shopping cart, and finally produces a list of recommended products.

In [26]:
# This function takes a cluster and the quantity of every product already purchased or in the user's cart
from pyspark.sql.functions import desc
def get_recommendations_by_cluster(cluster, purchased_quantities):
    # Existing customer products
    print('PRODUCTS ALREADY PURCHASED/IN CART:')
    customer_products = []
    for i in range(0, len(product_cols)):
        if purchased_quantities[i] > 0:
            customer_products.append((product_cols[i], purchased_quantities[i]))
    df_customer_products = sc.parallelize(customer_products).toDF(["PRODUCT","COUNT"])
    df_customer_products.show()
    # Get popular products in the cluster
    print('POPULAR PRODUCTS IN CLUSTER:')
    cluster_products = get_popular_products_in_cluster(cluster)
    df_cluster_products = sc.parallelize(cluster_products).toDF(["PRODUCT","COUNT"])
    df_cluster_products.show()
    # Filter out products the user has already purchased
    print('RECOMMENDED PRODUCTS:')
    df_recommended_products = df_cluster_products.alias('cl').join(df_customer_products.alias('cu'), df_cluster_products['PRODUCT'] == df_customer_products['PRODUCT'], 'leftouter')
    df_recommended_products = df_recommended_products.filter('cu.PRODUCT IS NULL').select('cl.PRODUCT','cl.COUNT').sort(desc('cl.COUNT'))
    df_recommended_products.show(10)

Next, run this cell to create a function that produces a list of recommended items based on the products and quantities in a user's cart. This function uses Watson Machine Learning to calculate the cluster based on the shopping cart contents and then calls the **get_recommendations_by_cluster** function.

In [27]:
# This function would be used to find recommendations based on the products and quantities in a user's cart
def get_recommendations_for_shopping_cart(products, quantities):
    fields = []
    values = []
    for product_col in product_cols:
        field = 'sum({})'.format(product_col)
        if product_col in products:
            value = quantities[products.index(product_col)]
        else:
            value = 0
        fields.append(field)
        values.append(value)
    return get_recommendations_by_cluster(get_cluster_from_watson_ml(fields, values), values)

Run this cell to create a function that produces a list of recommended items based on the purchase history of a customer. This function uses Watson Machine Learning to calculate the cluster based on the customer's purchase history and then calls the **get_recommendations_by_cluster** function.

In [28]:
# This function is used to find recommendations based on the purchase history of a customer
def get_recommendations_for_customer_purchase_history(customer_id):
    product_counts = get_product_counts_for_customer(customer_id)
    fields = product_counts[0]
    values = product_counts[1]
    return get_recommendations_by_cluster(get_cluster_from_watson_ml(fields, values), values)

Now you can take customer 12027 and produce a recommendation based on that customer's purchase history:

In [29]:
get_recommendations_for_customer_purchase_history(12027)

PRODUCTS ALREADY PURCHASED/IN CART:
+-------------+-----+
|      PRODUCT|COUNT|
+-------------+-----+
|      Diapers|    1|
|    Baby wash|    1|
|         Beer|    1|
|         Wine|    3|
|    Medicines|    3|
|       Cheese|    3|
| Frozen Foods|    1|
|Kitchen Items|    1|
|     Sea Food|    1|
|       Spices|    2|
+-------------+-----+

POPULAR PRODUCTS IN CLUSTER:
+-----------------+-----+
|          PRODUCT|COUNT|
+-----------------+-----+
|           Cheese|   74|
|        Medicines|   71|
|             Wine|   61|
|           Spices|   56|
|         Sea Food|   35|
|     Frozen Foods|   32|
|    Personal Care|   32|
|       Cigarettes|   29|
|       Condiments|   21|
|     Canned Foods|   14|
| Fresh Vegetables|   10|
|          Diapers|    9|
|Cleaning Products|    8|
|          Formula|    6|
|     Fresh Fruits|    6|
|        Baby Food|    5|
|             Beer|    5|
|           Lotion|    4|
|        Baby wash|    4|
|            Wipes|    4|
+-----------------+-----+
on

Now, take a sample shopping cart and produce a recommendation based on the items in the cart:

In [30]:
get_recommendations_for_shopping_cart(['Diapers','Baby wash','Oatmeal'],[1,2,1])

PRODUCTS ALREADY PURCHASED/IN CART:
+---------+-----+
|  PRODUCT|COUNT|
+---------+-----+
|  Diapers|    1|
|Baby wash|    2|
|  Oatmeal|    1|
+---------+-----+

POPULAR PRODUCTS IN CLUSTER:
+-----------------+-----+
|          PRODUCT|COUNT|
+-----------------+-----+
|          Diapers|   69|
|             Beer|   55|
|     Sports Drink|   15|
|          Popcorn|   15|
|       Condiments|   15|
|        Baby wash|   14|
|  Office Supplies|   14|
|           Lotion|   13|
|            Wipes|   11|
|        Club Soda|   11|
|          Oatmeal|   11|
|    Kitchen Items|   11|
|          Formula|    9|
|     Fresh Fruits|    8|
|Cleaning Products|    8|
|        Baby Food|    7|
| Fresh Vegetables|    7|
|     Canned Foods|    2|
|     Pet Supplies|    1|
+-----------------+-----+

RECOMMENDED PRODUCTS:
+---------------+-----+
|        PRODUCT|COUNT|
+---------------+-----+
|           Beer|   55|
|   Sports Drink|   15|
|     Condiments|   15|
|        Popcorn|   15|
|Office Supplies|  

The next optional section outlines how you can easily expose recommendations to notebook users, for example for test purposes.

<a id="test_recomm"></a>
### 6.1 Test product recommendations model

You can interactively test your product recommendations model using a simple PixieApp. [PixieApps](https://pixiedust.github.io/pixiedust/pixieapps.html) encapsulate business logic and data visualizations, making it easy for notebook users to explore data without having to write any code. Typically these applications are pre-packaged and imported into a notebook. However, for illustrative purposes we've embedded the product recommendation source code in this notebook.

![](https://raw.githubusercontent.com/IBM/product-recommendation-with-watson-ml/master/doc/source/images/product_recommendation_app.png")

Run the cell below, add items to the shopping cart and click the _Refresh_ button to review the recommendation results.

In [ ]:
# This function takes a cluster and the quantity of every product already purchased or in the user's cart & returns the data frame of recommendations for the PixieApp
from pyspark.sql.functions import desc
def get_recommendations_by_cluster_app(cluster, purchased_quantities):
    # Existing customer products
    customer_products = []
    for i in range(0, len(product_cols)):
        if purchased_quantities[i] > 0:
            customer_products.append((product_cols[i], purchased_quantities[i]))
    df_customer_products = sc.parallelize(customer_products).toDF(["PRODUCT","COUNT"])
    # Get popular products in the cluster
    cluster_products = get_popular_products_in_cluster(cluster)
    df_cluster_products = sc.parallelize(cluster_products).toDF(["PRODUCT","COUNT"])
    # Filter out products the user has already purchased
    df_recommended_products = df_cluster_products.alias('cl').join(df_customer_products.alias('cu'), df_cluster_products['PRODUCT'] == df_customer_products['PRODUCT'], 'leftouter')
    df_recommended_products = df_recommended_products.filter('cu.PRODUCT IS NULL').select('cl.PRODUCT','cl.COUNT').sort(desc('cl.COUNT'))
    return df_recommended_products


# PixieDust sample application

from pixiedust.display.app import *

@PixieApp
class RecommenderPixieApp:
    def setup(self):
        self.product_cols = product_cols
        
    def computeUserRecs(self, shoppingcart):   
        #format products and quantities from shopping cart display data
        lst = list(zip(*[(item.split(":")[0],int(item.split(":")[1])) for item in shoppingcart.split(",")]))
        products = list(lst[0])
        quantities = list(lst[1])
        #format for the Model function
        lst = list(zip(*[('sum({})'.format(item),quantities[products.index(item)] if item in products else 0) for item in self.product_cols]))
        fields = list(lst[0])
        values = list(lst[1])
        #call the run Model function
        recommendations_df = get_recommendations_by_cluster_app(get_cluster_from_watson_ml(fields, values), values)
        recs = [row["PRODUCT"] for row in recommendations_df.rdd.collect()]
        return recs[:5]
    
    @route(shoppingCart="*")
    def _recommendation(self, shoppingCart):
        recommendation = self.computeUserRecs(shoppingCart)
        self._addHTMLTemplateString(
        """
        <table style="width:100%"> {% for item in recommendation %} <tr> <td type="text" style="text-align:left">{{item}}</td> </tr> {% endfor %} </table>
        """, recommendation = recommendation)

        
    @route()
    def main(self):
        return """
        <script>
        function getValuesRec(){
            return $( "input[id^='prod']" )
            .filter(function( index ) {
                return parseInt($(this).val()) > 0;})
            .map(function(i, product) {
                return $(product).attr("name") + ":" + $(product).val();
            }).toArray().join(",");}
            
        function getValuesCart(){
            return $( "input[id^='prod']" )
            .filter(function( index ) {
                return parseInt($(this).val()) > 0; })
            .map(function(i, product) {
                return $(product).attr("name") + ":" + $(product).val();
            }).toArray(); }
        
        function populateCart(field) {
            user_cart = getValuesCart();
            $("#user_cart{{prefix}}").html("");
            if (user_cart.length > 0) {
                for (var i in user_cart) {
                    var item = user_cart[i];
                    var item_arr = item.split(":")
                    $("#user_cart{{prefix}}").append('<tr><td style="text-align:left">'+item_arr[1]+" "+item_arr[0]+"</td></tr>"); } }
            else { $("#user_cart{{prefix}}").append('<tr><td style="text-align:left">'+ "Cart Empty" +"</td></tr>"); } }
        
        function increase_by_one(field) {
            nr = parseInt(document.getElementById(field).value);
            document.getElementById(field).value = nr + 1;
            populateCart(field); }
        
        function decrease_by_one(field) {
            nr = parseInt(document.getElementById(field).value);
            if (nr > 0) { if( (nr - 1) >= 0) { document.getElementById(field).value = nr - 1; } }
            populateCart(field); } 
        </script>
        
        <table> Products: {% for item in this.product_cols %}
            {% if loop.index0 is divisibleby 4 %} <tr> {% endif %}
                <div class="prod-quantity">
                <td class="col-md-3">{{item}}:</td><td><input size="2" id="prod{{loop.index}}{{prefix}}" class="prods" type="text" 
                    style="text-align:center" value="0" name="{{item}}" /></td>
                <td><button onclick="increase_by_one('prod{{loop.index}}{{prefix}}');">+</button></td>
                <td><button onclick="decrease_by_one('prod{{loop.index}}{{prefix}}');">-</button></td>
                </div>
            {% if ((not loop.first) and (loop.index0 % 4 == 3)) or (loop.last) %} </tr> {% endif %}
        {% endfor %} </table>
        
        <div class="row">
            <div class="col-sm-6"> Your Cart: </div>
            <div class="col-sm-6"> Your Recommendations: <button pd_options="shoppingCart=$val(getValuesRec)" pd_target="recs{{prefix}}"> 
                <pd_script type="preRun"> if (getValuesRec()==""){alert("Your cart is empty");return false;} return true;
                </pd_script>Refresh </button> 
            </div>
        </div>
        
        <div class="row">
        <div class="col-sm-3"> <table style="width:100%" id="user_cart{{prefix}}"> </table> </div> <div class="col-sm-3"> </div>
        <div class="col-sm-3" id="recs{{prefix}}" pd_loading_msg="Calling your model in Watson ML"></div> <div class="col-sm-3"> </div>
        </div>
        """
        
    

#run the app
RecommenderPixieApp().run(runInDialog='false')

## <font color=green>Congratulations</font>, you've sucessfully created a recommendation engine, deployed it to the Watson Machine Learning service, and created a PixieApp!

You can now switch to the Watson Machine Learning console to deploy the model and then test it in application, or continue within the notebook to deploy the model using the APIs.